# Подготовка данных

In [1]:
import sqlite3

import pandas as pd
import numpy as np
import ta

Загрузка данных из БД созданной заранее (pj2)

In [2]:
# Установить соединение с базой данных
conn = sqlite3.connect(r'C:\Users\Alkor\gd\data_quote_db\MIX_futures_minute.db')

# SQL-запрос для извлечения данных
# query = "SELECT TRADEDATE, VOLUME FROM Minute"
query = "SELECT * FROM Minute"

# Загрузить данные в DataFrame
df = pd.read_sql_query(query, conn)

# Закрыть соединение
conn.close()

# Прописные в названиях колонок
df.columns = map(str.lower, df.columns)

df

,tradedate,secid,open,low,high,close,volume,lsttrade
0,2015-01-05 10:00:00,RIH5_2015,78450.0,74320.0,78450.0,74670.0,5970,2015-03-16
1,2015-01-05 10:01:00,RIH5_2015,74670.0,73520.0,74730.0,73580.0,3455,2015-03-16
2,2015-01-05 10:02:00,RIH5_2015,73560.0,73200.0,73570.0,73360.0,2721,2015-03-16
3,2015-01-05 10:03:00,RIH5_2015,73380.0,73250.0,74110.0,73670.0,4008,2015-03-16
4,2015-01-05 10:04:00,RIH5_2015,73650.0,73540.0,74000.0,73930.0,1471,2015-03-16
...,...,...,...,...,...,...,...,...
2037585,2024-12-30 23:45:00,RIH5,90450.0,90450.0,90470.0,90450.0,33,2025-03-20
2037586,2024-12-30 23:46:00,RIH5,90470.0,90450.0,90470.0,90450.0,83,2025-03-20
2037587,2024-12-30 23:47:00,RIH5,90450.0,90410.0,90450.0,90430.0,35,2025-03-20
2037588,2024-12-30 23:48:00,RIH5,90430.0,90420.0,90480.0,90480.0,176,2025-03-20


При просмотре данных заметил, что есть мусорные данные в начале торгового дня где open, high, low, close равны друг другу.  
Проверка таких данных.

In [3]:
# Преобразуем 'tradedate' в datetime формат, если это ещё не сделано
df['tradedate'] = pd.to_datetime(df['tradedate'])

# Фильтрация по условиям
filtered_df = df[
    # Все значения равны друг другу
    (df['open'] == df['high']) & 
    (df['open'] == df['low']) &
    (df['open'] == df['close']) &
    # Минуты равны 59
    (df['tradedate'].dt.minute == 59) &  
    # Первая строка за дату
    (df.groupby(df['tradedate'].dt.date)['tradedate'].rank(method='first') == 1)  
]

filtered_df

,tradedate,secid,open,low,high,close,volume,lsttrade
1675372,2023-04-03 08:59:00,RIM3,98700.0,98700.0,98700.0,98700.0,124,2023-06-15
1677910,2023-04-06 08:59:00,RIM3,97950.0,97950.0,97950.0,97950.0,29,2023-06-15
1679620,2023-04-10 08:59:00,RIM3,97650.0,97650.0,97650.0,97650.0,42,2023-06-15
1680468,2023-04-11 08:59:00,RIM3,98960.0,98960.0,98960.0,98960.0,14,2023-06-15
1681327,2023-04-12 08:59:00,RIM3,96900.0,96900.0,96900.0,96900.0,5,2023-06-15
...,...,...,...,...,...,...,...,...
2033602,2024-12-25 09:59:00,RIH5,85140.0,85140.0,85140.0,85140.0,2,2025-03-20
2034407,2024-12-26 09:59:00,RIH5,87410.0,87410.0,87410.0,87410.0,5,2025-03-20
2035216,2024-12-27 09:59:00,RIH5,88380.0,88380.0,88380.0,88380.0,11,2025-03-20
2036016,2024-12-28 09:59:00,RIH5,88850.0,88850.0,88850.0,88850.0,5,2025-03-20


Это премаркет который потом помешает при конвертации в 5 минутные бара. Удаляем эти строки.

In [4]:
df = (
    df.merge(filtered_df, how='outer', indicator=True)
    .query('_merge == "left_only"').drop(columns=['_merge'])
    )

df

,tradedate,secid,open,low,high,close,volume,lsttrade
0,2015-01-05 10:00:00,RIH5_2015,78450.0,74320.0,78450.0,74670.0,5970,2015-03-16
1,2015-01-05 10:01:00,RIH5_2015,74670.0,73520.0,74730.0,73580.0,3455,2015-03-16
2,2015-01-05 10:02:00,RIH5_2015,73560.0,73200.0,73570.0,73360.0,2721,2015-03-16
3,2015-01-05 10:03:00,RIH5_2015,73380.0,73250.0,74110.0,73670.0,4008,2015-03-16
4,2015-01-05 10:04:00,RIH5_2015,73650.0,73540.0,74000.0,73930.0,1471,2015-03-16
...,...,...,...,...,...,...,...,...
2037585,2024-12-30 23:45:00,RIH5,90450.0,90450.0,90470.0,90450.0,33,2025-03-20
2037586,2024-12-30 23:46:00,RIH5,90470.0,90450.0,90470.0,90450.0,83,2025-03-20
2037587,2024-12-30 23:47:00,RIH5,90450.0,90410.0,90450.0,90430.0,35,2025-03-20
2037588,2024-12-30 23:48:00,RIH5,90430.0,90420.0,90480.0,90480.0,176,2025-03-20


# Конвертирование 1 минутных котировок в 5 минутные

In [5]:
# Убедимся, что колонка  tradedate имеет тип datetime
df["tradedate"] = pd.to_datetime(df["tradedate"])

# Устанавливаем  tradedate как индекс
df.set_index("tradedate", inplace=True)

# Агрегирование до 5-минутных баров
df_m5 = df.resample("5min").agg({
    "open": "first",    # Первая цена в периоде
    "high": "max",      # Максимальная цена
    "low": "min",       # Минимальная цена
    "close": "last",    # Последняя цена
    "volume": "sum"    # Суммарный объём
})

# Сбрасываем индекс
df_m5 = df_m5.reset_index()

# Стереть строки с NaN
df_m5.dropna(inplace=True)

# Переиндексация
df_m5 = df_m5.reset_index(drop=True)

df_m5

,tradedate,open,high,low,close,volume
0,2015-01-05 10:00:00,78450.0,78450.0,73200.0,73930.0,17625
1,2015-01-05 10:05:00,73890.0,74700.0,73660.0,74700.0,7143
2,2015-01-05 10:10:00,74700.0,74720.0,73830.0,73950.0,4584
3,2015-01-05 10:15:00,73970.0,74440.0,73620.0,74430.0,4213
4,2015-01-05 10:20:00,74400.0,74720.0,74220.0,74410.0,4498
...,...,...,...,...,...,...
411716,2024-12-30 23:25:00,90630.0,90740.0,90350.0,90390.0,415
411717,2024-12-30 23:30:00,90430.0,90430.0,90380.0,90380.0,89
411718,2024-12-30 23:35:00,90380.0,90390.0,90280.0,90390.0,170
411719,2024-12-30 23:40:00,90390.0,90480.0,90390.0,90450.0,615


### TVI

In [6]:
def calculate_ema(series, period):
    """
    Вычисляет экспоненциальное скользящее среднее (EMA) 
    для указанного ряда данных.
    """
    return series.ewm(span=period, adjust=False).mean()

def add_tvi_column(df, r=12, s=12, u=5, point=0.0001):
    """
    Добавляет колонку TVI в DataFrame.
    
    :param df: DataFrame с колонками ['open', 'close', 'volume']
    :param r: Период для EMA для UpTicks и DownTicks
    :param s: Период для второго EMA
    :param u: Период для EMA на TVI_calculate
    :param point: Минимальное изменение цены (параметр MyPoint в MQL4)
    :return: DataFrame с добавленной колонкой 'TVI'
    """
    # Проверяем, что необходимые колонки существуют
    required_columns = ['open', 'close', 'volume']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"DataFrame должен содержать колонку '{col}'")

    # Вычисляем UpTicks и DownTicks
    df['UpTicks'] = (df['volume'] + (df['close'] - df['open']) / point) / 2
    df['DownTicks'] = df['volume'] - df['UpTicks']

    # EMA для UpTicks и DownTicks
    df['EMA_UpTicks'] = calculate_ema(df['UpTicks'], r)
    df['EMA_DownTicks'] = calculate_ema(df['DownTicks'], r)

    # Второе EMA для UpTicks и DownTicks
    df['DEMA_UpTicks'] = calculate_ema(df['EMA_UpTicks'], s)
    df['DEMA_DownTicks'] = calculate_ema(df['EMA_DownTicks'], s)

    # Расчет TVI_calculate
    df['TVI_calculate'] = 100.0 * (df['DEMA_UpTicks'] - df['DEMA_DownTicks']) / (
        df['DEMA_UpTicks'] + df['DEMA_DownTicks']
    )

    # EMA на TVI_calculate
    df['TVI'] = calculate_ema(df['TVI_calculate'], u)

    # Удаляем промежуточные колонки, если они не нужны
    df.drop(
        columns=['UpTicks', 'DownTicks', 'EMA_UpTicks', 'EMA_DownTicks', 
        'DEMA_UpTicks', 'DEMA_DownTicks', 'TVI_calculate'], inplace=True
        )

    return df

# Добавляем колонку TVI
df_m5 = add_tvi_column(df_m5, r=12, s=12, u=5, point=0.1)

# Результат
df_m5

,tradedate,open,high,low,close,volume,TVI
0,2015-01-05 10:00:00,78450.0,78450.0,73200.0,73930.0,17625,-256.453901
1,2015-01-05 10:05:00,73890.0,74700.0,73660.0,74700.0,7143,-255.254435
2,2015-01-05 10:10:00,74700.0,74720.0,73830.0,73950.0,4584,-253.367957
3,2015-01-05 10:15:00,73970.0,74440.0,73620.0,74430.0,4213,-250.544546
4,2015-01-05 10:20:00,74400.0,74720.0,74220.0,74410.0,4498,-246.875928
...,...,...,...,...,...,...,...
411716,2024-12-30 23:25:00,90630.0,90740.0,90350.0,90390.0,415,59.782874
411717,2024-12-30 23:30:00,90430.0,90430.0,90380.0,90380.0,89,50.252201
411718,2024-12-30 23:35:00,90380.0,90390.0,90280.0,90390.0,170,38.734820
411719,2024-12-30 23:40:00,90390.0,90480.0,90390.0,90450.0,615,29.783669


## GannHiLo

In [7]:
def calculate_sma(series, period):
    """
    Вычисляет простую скользящую среднюю (SMA).
    """
    return series.rolling(window=period, min_periods=1).mean()

def add_gann_hilo_column_optimized(df, period=10):
    """
    Оптимизированная версия добавления индикатора GannHiLo в DataFrame.
    
    :param df: DataFrame с колонками ['close', 'high', 'low']
    :param period: Период для расчета SMA
    :return: DataFrame с добавленными колонками 'Gann_HiLo'
    """
    # Проверка на необходимые колонки
    required_columns = ['close', 'high', 'low']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(
            f"DataFrame должен содержать колонки: {required_columns}"
            )

    # Расчет SMA для high и low
    sma_high = calculate_sma(df['high'], period)
    sma_low = calculate_sma(df['low'], period)

    # Вычисление GannHiLo векторным способом
    gann_hilo = (
        (df['close'] > sma_high.shift(1)).astype(int) - 
        (df['close'] < sma_low.shift(1)).astype(int)
        )

    # Добавляем колонку Gann_HiLo в DataFrame
    df['Gann_HiLo'] = gann_hilo

    return df

# Добавляем GannHiLo индикатор
df_m5 = add_gann_hilo_column_optimized(df_m5, period=10)

# Результат
df_m5


,tradedate,open,high,low,close,volume,TVI,Gann_HiLo
0,2015-01-05 10:00:00,78450.0,78450.0,73200.0,73930.0,17625,-256.453901,0
1,2015-01-05 10:05:00,73890.0,74700.0,73660.0,74700.0,7143,-255.254435,0
2,2015-01-05 10:10:00,74700.0,74720.0,73830.0,73950.0,4584,-253.367957,0
3,2015-01-05 10:15:00,73970.0,74440.0,73620.0,74430.0,4213,-250.544546,0
4,2015-01-05 10:20:00,74400.0,74720.0,74220.0,74410.0,4498,-246.875928,0
...,...,...,...,...,...,...,...,...
411716,2024-12-30 23:25:00,90630.0,90740.0,90350.0,90390.0,415,59.782874,-1
411717,2024-12-30 23:30:00,90430.0,90430.0,90380.0,90380.0,89,50.252201,-1
411718,2024-12-30 23:35:00,90380.0,90390.0,90280.0,90390.0,170,38.734820,-1
411719,2024-12-30 23:40:00,90390.0,90480.0,90390.0,90450.0,615,29.783669,0


## T3 (T3ma) (8, 0.618)

In [8]:
def calculate_t3ma(df, period=8, b=0.618):
    """
    Рассчитывает индикатор T3MA и добавляет его как новую колонку в DataFrame.

    :param df: DataFrame с колонкой 'close'
    :param period: Период T3MA
    :param b: Параметр T3MA (значение сглаживания)
    :return: DataFrame с добавленной колонкой 'T3MA'
    """
    # Проверка на наличие колонки 'close'
    if 'close' not in df.columns:
        raise ValueError("DataFrame должен содержать колонку 'close'")

    close = df['close'].values

    # Коэффициенты
    b2 = b ** 2
    b3 = b2 * b
    c1 = -b3
    c2 = 3 * (b2 + b3)
    c3 = -3 * (2 * b2 + b + b3)
    c4 = 1 + 3 * b + b3 + 3 * b2
    n = max(1, 1 + 0.5 * (period - 1))
    w1 = 2 / (n + 1)
    w2 = 1 - w1

    # Создание временных массивов
    e1 = np.zeros_like(close)
    e2 = np.zeros_like(close)
    e3 = np.zeros_like(close)
    e4 = np.zeros_like(close)
    e5 = np.zeros_like(close)
    e6 = np.zeros_like(close)

    # Векторизованный расчет экспоненциальных сглаживаний
    e1[0] = close[0]
    for i in range(1, len(close)):
        e1[i] = w1 * close[i] + w2 * e1[i - 1]

    e2[0] = e1[0]
    for i in range(1, len(e1)):
        e2[i] = w1 * e1[i] + w2 * e2[i - 1]

    e3[0] = e2[0]
    for i in range(1, len(e2)):
        e3[i] = w1 * e2[i] + w2 * e3[i - 1]

    e4[0] = e3[0]
    for i in range(1, len(e3)):
        e4[i] = w1 * e3[i] + w2 * e4[i - 1]

    e5[0] = e4[0]
    for i in range(1, len(e4)):
        e5[i] = w1 * e4[i] + w2 * e5[i - 1]

    e6[0] = e5[0]
    for i in range(1, len(e5)):
        e6[i] = w1 * e5[i] + w2 * e6[i - 1]

    # Итоговый расчет T3
    t3 = c1 * e6 + c2 * e5 + c3 * e4 + c4 * e3

    # Добавление результата в DataFrame
    df['T3MA'] = t3
    return df

# Рассчитываем T3MA и добавляем в DataFrame
df_m5 = calculate_t3ma(df_m5, period=8, b=0.618)

# Результат
df_m5

,tradedate,open,high,low,close,volume,TVI,Gann_HiLo,T3MA
0,2015-01-05 10:00:00,78450.0,78450.0,73200.0,73930.0,17625,-256.453901,0,73930.000000
1,2015-01-05 10:05:00,73890.0,74700.0,73660.0,74700.0,7143,-255.254435,0,74030.138493
2,2015-01-05 10:10:00,74700.0,74720.0,73830.0,73950.0,4584,-253.367957,0,74092.939249
3,2015-01-05 10:15:00,73970.0,74440.0,73620.0,74430.0,4213,-250.544546,0,74167.176629
4,2015-01-05 10:20:00,74400.0,74720.0,74220.0,74410.0,4498,-246.875928,0,74244.053078
...,...,...,...,...,...,...,...,...,...
411716,2024-12-30 23:25:00,90630.0,90740.0,90350.0,90390.0,415,59.782874,-1,90522.395153
411717,2024-12-30 23:30:00,90430.0,90430.0,90380.0,90380.0,89,50.252201,-1,90499.252826
411718,2024-12-30 23:35:00,90380.0,90390.0,90280.0,90390.0,170,38.734820,-1,90469.233842
411719,2024-12-30 23:40:00,90390.0,90480.0,90390.0,90450.0,615,29.783669,0,90448.875367


# CCI (20)

In [9]:
# Вычисление CCI с периодом 20
df_m5['CCI_20'] = ta.trend.cci(
    high=df_m5['high'], low=df_m5['low'], close=df_m5['close'], window=20
    )

df_m5


,tradedate,open,high,low,close,volume,TVI,Gann_HiLo,T3MA,CCI_20
0,2015-01-05 10:00:00,78450.0,78450.0,73200.0,73930.0,17625,-256.453901,0,73930.000000,NaN
1,2015-01-05 10:05:00,73890.0,74700.0,73660.0,74700.0,7143,-255.254435,0,74030.138493,NaN
2,2015-01-05 10:10:00,74700.0,74720.0,73830.0,73950.0,4584,-253.367957,0,74092.939249,NaN
3,2015-01-05 10:15:00,73970.0,74440.0,73620.0,74430.0,4213,-250.544546,0,74167.176629,NaN
4,2015-01-05 10:20:00,74400.0,74720.0,74220.0,74410.0,4498,-246.875928,0,74244.053078,NaN
...,...,...,...,...,...,...,...,...,...,...
411716,2024-12-30 23:25:00,90630.0,90740.0,90350.0,90390.0,415,59.782874,-1,90522.395153,70.235819
411717,2024-12-30 23:30:00,90430.0,90430.0,90380.0,90380.0,89,50.252201,-1,90499.252826,-7.526882
411718,2024-12-30 23:35:00,90380.0,90390.0,90280.0,90390.0,170,38.734820,-1,90469.233842,-43.360434
411719,2024-12-30 23:40:00,90390.0,90480.0,90390.0,90450.0,615,29.783669,0,90448.875367,23.700624


# Сигналы индикаторов

In [10]:
# Добавляем новую колонку с сигналом TVI
df_m5['TVI_S'] = df_m5['TVI'].diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

# Добавляем новую колонку с сигналом T3MA
df_m5['T3_S'] = df_m5['T3MA'].diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

# Добавляем новую колонку с сигналом от CCI
df_m5['CCI_S'] = df_m5['CCI_20'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

df_m5 = df_m5[['tradedate', 'open', 'high', 'low', 'close', 
               'TVI_S', 'CCI_S', 'T3_S', 'Gann_HiLo']]

df_m5 = df_m5.copy()

# Переименование колонок
df_m5.rename(
    columns={'TVI_S': 'tvi', 'CCI_S': 'cci', 'T3_S': 't3', 'Gann_HiLo': 'ghl'}, 
    inplace=True
    )

df_m5

,tradedate,open,high,low,close,tvi,cci,t3,ghl
0,2015-01-05 10:00:00,78450.0,78450.0,73200.0,73930.0,0,0,0,0
1,2015-01-05 10:05:00,73890.0,74700.0,73660.0,74700.0,1,0,1,0
2,2015-01-05 10:10:00,74700.0,74720.0,73830.0,73950.0,1,0,1,0
3,2015-01-05 10:15:00,73970.0,74440.0,73620.0,74430.0,1,0,1,0
4,2015-01-05 10:20:00,74400.0,74720.0,74220.0,74410.0,1,0,1,0
...,...,...,...,...,...,...,...,...,...
411716,2024-12-30 23:25:00,90630.0,90740.0,90350.0,90390.0,-1,1,-1,-1
411717,2024-12-30 23:30:00,90430.0,90430.0,90380.0,90380.0,-1,-1,-1,-1
411718,2024-12-30 23:35:00,90380.0,90390.0,90280.0,90390.0,-1,-1,-1,-1
411719,2024-12-30 23:40:00,90390.0,90480.0,90390.0,90450.0,-1,1,-1,0


### Сохранение в файл

In [11]:
df_m5.to_csv("df_m5_TVI_CCI_T3_GHL.csv", index=False)